In [2]:
import sqlalchemy as db

engine = db.create_engine('sqlite:///mydb.db', connect_args={'timeout': 100}) #is this address a default for customized db?
conn = engine.connect()
metadata = db.MetaData()

In [3]:
passr = db.Table(
    'Passengers', metadata,
    db.Column('Pass_id', db.Integer, primary_key = True),
    db.Column('Name', db.Text, nullable = False),
    db.Column('Gender', db.Text, nullable = False),
    db.Column('Age', db.Integer, nullable = False)
)

tickets = db.Table(
    'Tickets', metadata,
    db.Column('Ticket_num', db.Integer, primary_key = True),
    db.Column('Pass_id', db.Integer, db.ForeignKey('Passengers.Pass_id'), nullable = False),
    db.Column('Price', db.Integer, nullable = False),
)

cabins = db.Table(
    'Cabins', metadata,
    db.Column('Flight_id', db.Integer, db.ForeignKey('Flights.Flight_id'), nullable = False),
    db.Column('Ticket_num', db.Integer, db.ForeignKey('Tickets.Ticket_num'), nullable = False),
    db.Column('Seat', db.Text(3), nullable = False)
)

flights = db.Table(
    'Flights', metadata,
    db.Column('Flight_id', db.Integer, primary_key = True),
    db.Column('Flight_num', db.Text(6), nullable = False),
    db.Column('Destination', db.Text(15), nullable = False),
    db.Column('Departure_date', db.Text, nullable = False )
)

metadata.create_all(engine)

In [4]:
insertion_1 = passr.insert().values([
    {'Name': 'Bruce Lee','Gender': 'male', 'Age': 35},
    {'Pass_id': 1,'Name': 'Jeniffer Aniston','Gender': 'female', 'Age': 28},
    {'Pass_id': 2,'Name': 'Angelina Jolie','Gender': 'female', 'Age': 19},
    {'Pass_id': 3,'Name': 'Jonny Depp','Gender': 'male', 'Age': 41},
    {'Pass_id': 4,'Name': 'Stan Lee','Gender': 'male', 'Age': 62}
])

insertion_2 = tickets.insert().values([
    {'Pass_id': 2, 'Price': 100},
    {'Pass_id': 1, 'Price': 2500},
    {'Pass_id': 3, 'Price': 460},
    {'Pass_id': 2, 'Price': 75},
    {'Pass_id': 5, 'Price': 500}
])

insertion_3 = cabins.insert().values([
    {'Flight_id': 1, 'Ticket_num': 1, 'Seat': '2A'},
    {'Flight_id': 2, 'Ticket_num': 2, 'Seat': '21C'},
    {'Flight_id': 2, 'Ticket_num': 3, 'Seat': '16E'},
    {'Flight_id': 1, 'Ticket_num': 5, 'Seat': '4B'}
])

insertion_4 = flights.insert().values([
    {'Flight_num': '175NY', 'Destination': 'New York', 'Departure_date': '01/02/2003'},
    {'Flight_num': '222QA', 'Destination': 'Quatar', 'Departure_date': '04/12/2004'},
    {'Flight_num': '356PA', 'Destination': 'Paris', 'Departure_date': '05/08/2003'}
])

In [5]:
conn.execute(insertion_1)
conn.execute(insertion_2)
conn.execute(insertion_3)
conn.execute(insertion_4)

In [6]:
conn.exec_driver_sql('SELECT *'
                      'FROM Passengers p JOIN Tickets t ON p.Pass_id = t.Pass_id'
).fetchall()

[(2, 'Jeniffer Aniston', 'female', 28, 1, 2, 100),
 (1, 'Bruce Lee', 'male', 35, 2, 1, 2500),
 (3, 'Angelina Jolie', 'female', 19, 3, 3, 460),
 (2, 'Jeniffer Aniston', 'female', 28, 4, 2, 75),
 (5, 'Stan Lee', 'male', 62, 5, 5, 500)]

In [7]:
conn.exec_driver_sql('SELECT *'
                    'FROM Passengers p LEFT JOIN Tickets t ON p.Pass_id = t.Pass_id'
).fetchall()

[(1, 'Bruce Lee', 'male', 35, 2, 1, 2500),
 (2, 'Jeniffer Aniston', 'female', 28, 4, 2, 75),
 (2, 'Jeniffer Aniston', 'female', 28, 1, 2, 100),
 (3, 'Angelina Jolie', 'female', 19, 3, 3, 460),
 (4, 'Jonny Depp', 'male', 41, None, None, None),
 (5, 'Stan Lee', 'male', 62, 5, 5, 500)]

In [8]:
conn.exec_driver_sql('SELECT f.Flight_num, COUNT(t.Pass_id) '
                      'FROM Tickets t JOIN Cabins c ON t.Ticket_num = c.Ticket_num '
                      'JOIN Flights f ON c.Flight_id = f.Flight_id '
                      'JOIN Passengers p ON p.Pass_id = t.Pass_id '
                      'GROUP BY f.Flight_num '
                      'HAVING COUNT(t.Pass_id) > 1'
).fetchall()

[('175NY', 2), ('222QA', 2)]

In [9]:
more_pass = db.Table(
    'Passengers_ext', metadata,
    db.Column('Name', db.Text),
    db.Column('Age', db.Integer)
)
metadata.create_all(engine)

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: 
CREATE TABLE "Passengers_ext" (
	"Name" TEXT, 
	"Age" INTEGER
)

]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [1]:
pass_ext = more_pass.insert().values([
    {'Name': 'Paris Hilton', 'Age': 17},
    {'Name': 'Bruce Lee', 'Age': 35}
])


NameError: name 'more_pass' is not defined